In [1]:
import numpy as np
from cvxopt import matrix
from cvxopt import solvers

solvers.options['show_progress'] = False

In [2]:
indicator = lambda exp: 1 if exp else 0

def gaussian_kernel(x, z, gamma):
    return np.exp(-gamma * np.linalg.norm(x-z)**2)

def extract_data(data):
    """
        Returns the feature data size m, feature vector x and label vector y.
    """
    m = data.shape[0]
    x = data[:, :-1]  # features
    x /= x.max()    # scale to 0-1
    y = data[:, -1].reshape((m, 1))  # labels

    return m, x, y

def get_class_data(data, class1, class2):
    """
        Returns data for class labels class1 and class2. Also changes the label class1 to -1 and class2 to 1 in the data.
    """
    data = data[(data[:,-1] == class1) | (data[:, -1] == class2)] # filter for class1 and class2
    data[:, -1] = np.where(data[:, -1] == class1, -1, 1) # change class labels to -1 and 1
    return data

In [100]:
class SVMBinaryClassifier:

    def __init__(self, train_data, C, gamma):
        self.m, self.x, self.y = extract_data(train_data)
        self.C = C
        self.gamma = gamma

    def train(self):
        m, x, y, C, gamma = self.m, self.x, self.y, self.C, self.gamma
        
        P = np.zeros((m, m))

        for i in range(m):
            for j in range(i, m):
                P[i, j] = y[i] * y[j] * gaussian_kernel(x[i], x[j], gamma)
                P[j, i] = P[i, j]

        q = -np.ones((m, 1))
        G = np.vstack((-np.identity(m), np.identity(m)))
        h = np.vstack((np.zeros((m, 1)), np.full((m, 1), C)))
        A = y.T
        b = np.zeros(1)

        P = matrix(P, tc='d')
        q = matrix(q, tc='d')
        G = matrix(G, tc='d')
        h = matrix(h, tc='d')
        A = matrix(A, tc='d')
        b = matrix(b, tc='d')

        sol = solvers.qp(P,q,G,h,A,b)

        alpha = np.array(sol['x'])

        S = set() # support vectors

        # check for non-zero (>= epsilon) vectors
        epsilon = 1e-4
        while(len(S) == 0):
            for i in range(m):
                if alpha[i] >= epsilon:
                    S.add(i)
            epsilon = epsilon / 20

        b = sum(y[s] - sum(alpha[j] * y[j] * gaussian_kernel(x[j], x[s], gamma) for j in S) for s in S) / len(S) # take average over all support vectors

        self.alpha = alpha
        self.b = b

    def predict(self, x_predict):
        m, x, y, gamma, alpha, b = self.m, self.x, self.y, self.gamma, self.alpha, self.b
        return np.sign(sum(alpha[i] * y[i] * gaussian_kernel(x[i], x_predict, gamma) for i in range(m)) + b)


In [106]:
class SVMMultiClassifier:
    def __init__(self, total_classes, train_data, C, gamma):
        self.total_classes = total_classes
        self.svm = [[None for j in range(total_classes)] for i in range(total_classes)]
        for i in range(total_classes):
            for j in range(i+1, total_classes):
                self.svm[i][j] = SVMBinaryClassifier(get_class_data(train_data, i, j), C, gamma)

    def get_svms(self):
        return self.svm

    def train(self):
        total_classes, svm = self.total_classes, self.svm
        for i in range(total_classes):
            for j in range(i+1, total_classes):
                svm[i][j].train()

    def predict(self, x_predict):
        total_classes, svm = self.total_classes, self.svm
        votes = np.zeros(total_classes, np.int)
        for i in range(total_classes):
            for j in range(i+1, total_classes):
                p = svm[i][j].predict(x_predict)
                if p < 0:
                    votes[i] += 1
                else:
                    votes[j] += 1
        return np.argmax(votes)

In [96]:
train_file = "C:/IITD/sem5/col774-ml/datasets/fmnist_data/fashion_mnist/train.csv"
test_file = "C:/IITD/sem5/col774-ml/datasets/fmnist_data/fashion_mnist/test.csv"
val_file = "C:/IITD/sem5/col774-ml/datasets/fmnist_data/fashion_mnist/val.csv"

train_data = np.genfromtxt(train_file, delimiter=',')
test_data = np.genfromtxt(test_file, delimiter=',')
val_data = np.genfromtxt(val_file, delimiter=',')

In [19]:
train_data = train_data[:int(len(train_data)/5)]

train_data

array([[0., 0., 0., ..., 0., 0., 5.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 7.],
       ...,
       [0., 0., 0., ..., 0., 0., 7.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 8.]])

In [20]:
total_classes = 10
svm = [[None for j in range(total_classes)] for i in range(total_classes)]

import time
t0 = time.time()

for i in range(total_classes):
    for j in range(i+1, total_classes):
        print("------------svm------------", i, j)
        svm[i][j] = SVMBinaryClassifier(get_class_data(train_data, i, j), 1, 0.05)
        svm[i][j].train()
        t1 = time.time()
        print(t1 - t0)

------------svm------------ 0 1
0
5.188998460769653
------------svm------------ 0 2
0
11.514792919158936
------------svm------------ 0 3
0
18.2648286819458
------------svm------------ 0 4
0
24.259828329086304
------------svm------------ 0 5
0
29.937828302383423
------------svm------------ 0 6
0
37.76005983352661
------------svm------------ 0 7
0
42.84405851364136
------------svm------------ 0 8
0
49.49205994606018
------------svm------------ 0 9
0
54.69030451774597
------------svm------------ 1 2
0
60.339306592941284
------------svm------------ 1 3
0
66.4713044166565
------------svm------------ 1 4
0
72.57662677764893
------------svm------------ 1 5
0
78.3366596698761
------------svm------------ 1 6
0
84.31562829017639
------------svm------------ 1 7
0
89.94563007354736
------------svm------------ 1 8
0
96.44270634651184
------------svm------------ 1 9
0
101.92870283126831
------------svm------------ 2 3
0
108.70074105262756
------------svm------------ 2 4
0
118.55525183677673
--------

In [67]:
m_test, x_test, y_test = extract_data(test_data[:400])

print(m_test)

flag = False
test_predictions = np.zeros(m_test, np.int)

for t in range(m_test):
    if t % 50 == 0:
        print(t)
    votes = np.zeros(10, np.int)
    for i in range(total_classes):
        for j in range(i+1, total_classes):
            p = svm[i][j].predict(x_test[t])
            if p < 0:
                votes[i] += 1
            else:
                votes[j] += 1
    test_predictions[t] = np.argmax(votes)

# print(y_test[5], svm[4][6].predict(x_test[5]))

400
0
50
100
150
200
250
300
350


In [69]:
ff = (test_predictions == y_test.T)[0]
np.sum(ff) / 400

0.815

In [70]:
flag

False

In [72]:
# confusion matrix
confusion_matrix = np.zeros((10, 10), np.int)

for i in range(m_test):
    confusion_matrix[test_predictions[i]][int(y_test[i])] += 1

print(confusion_matrix)

[[31  0  0  2  0  0  4  0  0  0]
 [ 0 41  0  0  0  0  0  0  0  0]
 [ 1  0 27  0  2  0  3  0  0  0]
 [ 1  3  1 38  2  0  2  0  0  0]
 [ 0  0  7  0 25  0  4  0  0  0]
 [ 0  0  0  0  0 37  0  1  1  3]
 [11  0  6  3  1  0 22  0  1  0]
 [ 0  0  0  0  0  3  0 29  0  1]
 [ 2  0  2  1  0  1  0  0 36  1]
 [ 0  0  0  0  0  1  0  3  0 40]]


In [78]:
train_data_2 = train_data[:int(len(train_data)/5)]
rr = np.split(train_data_2, [int(len(train_data_2)/5)])
# rr[0].shape
rr[1].shape

(3600, 785)

In [101]:
# Validation

train_data = np.genfromtxt(train_file, delimiter=',')

train_data_2 = train_data[:int(len(train_data)/20)]
train_data_split = np.split(train_data_2, [int(len(train_data_2)/5)])
val_set_data = train_data_split[0]
train_set_data = train_data_split[1]

m_val, x_val, y_val = extract_data(val_set_data)

for C in [1e-3, 1, 5, 10]:
    print("C", C)
    svm = [[None for j in range(total_classes)] for i in range(total_classes)]
    for i in range(total_classes):
        for j in range(i+1, total_classes):
            # print("------------svm------------", i, j)
            svm[i][j] = SVMBinaryClassifier(get_class_data(train_set_data, i, j), C, 0.05)
            svm[i][j].train()


    val_predictions = np.zeros(m_val, np.int)

    for t in range(m_val):
        votes = np.zeros(10, np.int)
        for i in range(total_classes):
            for j in range(i+1, total_classes):
                p = svm[i][j].predict(x_val[t])
                if p < 0:
                    votes[i] += 1
                else:
                    votes[j] += 1
        val_predictions[t] = np.argmax(votes)

    # print(val_predictions, y_val.T, (val_predictions == y_val.T[0]))
    # acc = sum(indicator(val_predictions[p] == y_val[p]) for p in range(m_val)) / m_val
    acc = np.sum((val_predictions == y_val.T)[0]) / m_val
    print("acc", C, acc)

# print(val_data.shape, train_data.shape)

C 0.001
acc 0.001 0.09777777777777778
C 1
acc 1 0.76
C 5
acc 5 0.7511111111111111
C 10
acc 10 0.7511111111111111


In [107]:
# Validation

train_data = np.genfromtxt(train_file, delimiter=',')

train_data_2 = train_data[:int(len(train_data)/20)]
train_data_split = np.split(train_data_2, [int(len(train_data_2)/5)])
val_set_data = train_data_split[0]
train_set_data = train_data_split[1]

m_val, x_val, y_val = extract_data(val_set_data)

for C in [1e-3, 1, 5, 10]:
    print("C", C)
    svm_multi = SVMMultiClassifier(10, train_set_data, C, 0.05)
    # print(svm_multi.get_svms())
    svm_multi.train()

    val_predictions = np.zeros(m_val, np.int)

    for t in range(m_val):
        val_predictions[t] = svm_multi.predict(x_val[t])

    # print(val_predictions, y_val.T, (val_predictions == y_val.T[0]))
    # acc = sum(indicator(val_predictions[p] == y_val[p]) for p in range(m_val)) / m_val
    acc = np.sum((val_predictions == y_val.T)[0]) / m_val
    print("acc", C, acc)

# print(val_data.shape, train_data.shape)

C 0.001
acc 0.001 0.09777777777777778
C 1
acc 1 0.76
C 5
acc 5 0.7511111111111111
C 10
acc 10 0.7511111111111111
